In [1]:
# Install and import required libraries
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
packages = ['ipywidgets', 'matplotlib']
for package in packages:
    install_if_missing(package)

print("✅ All packages ready!")


✅ All packages ready!


In [2]:
# Import required libraries
import gymnasium as gym
from stable_baselines3 import SAC, TD3, A2C
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time
import os
import threading
import subprocess
from collections import deque
import warnings
warnings.filterwarnings('ignore')

print("📦 Libraries imported successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"📁 Current directory: {os.getcwd()}")


📦 Libraries imported successfully!
🐍 Python version: 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]
📁 Current directory: /Users/arhaan17/Coding/Movement_Tracking_Mujoco/test3


In [3]:
def execute_sb3_command(model_name, environment="Humanoid-v4", algorithm="A2C", show_available=True):
    """Execute sb3.py with specified parameters"""
    model_path = f"models/{model_name}.zip"
    
    print(f"🚀 Executing: python sb3.py {environment} {algorithm} -s {model_path}")
    print(f"📊 Model: {model_name}")
    print(f"🎮 Environment: {environment}")
    print(f"🤖 Algorithm: {algorithm}")
    print("\n" + "="*60)
    
    # Check if model exists
    if not os.path.exists(model_path):
        print(f"❌ Model not found: {model_path}")
        if show_available and os.path.exists("models"):
            print("\n📁 Available models:")
            models = sorted([f for f in os.listdir("models") if f.endswith('.zip')])
            for i, f in enumerate(models[:20], 1):  # Show first 20
                print(f"   {i:2}. {f}")
            if len(models) > 20:
                print(f"   ... and {len(models)-20} more models")
        return False
    
    try:
        print("🎮 Starting Mujoco viewer...")
        print("⚠️  The viewer will open in a new window")
        print("⏹️  Close the viewer window when done")
        print("🔄  Output will appear below...\n")
        
        # Execute the command with real-time output
        process = subprocess.Popen(
            ["python", "sb3.py", environment, algorithm, "-s", model_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True
        )
        
        # Print output in real-time
        output_lines = []
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                line = output.strip()
                print(line)
                output_lines.append(line)
        
        rc = process.poll()
        if rc == 0:
            print("\n✅ Execution completed successfully!")
            return True
        else:
            print(f"\n❌ Execution failed with return code: {rc}")
            return False
            
    except KeyboardInterrupt:
        print("\n⏹️ Execution interrupted by user")
        try:
            process.terminate()
        except:
            pass
        return False
    except Exception as e:
        print(f"💥 Error: {str(e)}")
        return False

# ========== MODIFY THESE PARAMETERS ==========
MODEL_NAME = "A2C_8125000"      # Change this to your desired model
ALGORITHM = "A2C"             # A2C, SAC, or TD3
ENVIRONMENT = "Humanoid-v4"   # Environment name
# =============================================

print(f"🎯 Quick Execution Settings:")
print(f"   📊 Model: {MODEL_NAME}")
print(f"   🤖 Algorithm: {ALGORITHM}")
print(f"   🎮 Environment: {ENVIRONMENT}")
print(f"\n📝 To change settings: Edit the parameters above and re-run this cell")
print(f"🚀 To execute: Run the next cell!")


🎯 Quick Execution Settings:
   📊 Model: A2C_8125000
   🤖 Algorithm: A2C
   🎮 Environment: Humanoid-v4

📝 To change settings: Edit the parameters above and re-run this cell
🚀 To execute: Run the next cell!


In [4]:
# Execute the command with current settings
print("🚀 Executing sb3.py command...\n")
success = execute_sb3_command(MODEL_NAME, ENVIRONMENT, ALGORITHM)

if success:
    print("\n🎉 Command executed successfully!")
else:
    print("\n⚠️  Command failed or was interrupted.")
    print("💡 Tips:")
    print("   - Check that the model file exists in models/ directory")
    print("   - Verify the algorithm matches your model type")
    print("   - Make sure sb3.py is in the current directory")


🚀 Executing sb3.py command...

🚀 Executing: python sb3.py Humanoid-v4 A2C -s models/A2C_8125000.zip
📊 Model: A2C_8125000
🎮 Environment: Humanoid-v4
🤖 Algorithm: A2C

🎮 Starting Mujoco viewer...
⚠️  The viewer will open in a new window
⏹️  Close the viewer window when done
🔄  Output will appear below...

/opt/homebrew/Caskroom/miniconda/base/envs/test3/lib/python3.10/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Humanoid-v4 is out of date. You should consider upgrading to version `v5`.
logger.deprecation(
/opt/homebrew/Caskroom/miniconda/base/envs/test3/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
warnings.warn(
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
/opt/homebrew/Caskroom/miniconda/base/envs/test3/lib/pyth

In [5]:
class EnhancedMujocoViewer:
    def __init__(self):
        self.current_process = None
        self.setup_ui()
        
    def get_available_models(self):
        """Get available models organized by algorithm"""
        models = {'A2C': [], 'SAC': [], 'TD3': []}
        
        if not os.path.exists('models'):
            print("⚠️  Models directory not found")
            return models
            
        for file in os.listdir('models'):
            if file.endswith('.zip'):
                for algo in models.keys():
                    if file.startswith(algo):
                        try:
                            # Extract training steps from filename
                            steps_str = file.replace(f'{algo}_', '').replace('.zip', '')
                            steps = int(steps_str)
                            models[algo].append((steps, file.replace('.zip', '')))
                        except:
                            models[algo].append((0, file.replace('.zip', '')))
        
        # Sort by training steps (descending)
        for algo in models:
            models[algo].sort(reverse=True)
            
        return models
    
    def update_models(self, change):
        """Update model dropdown when algorithm changes"""
        models = self.get_available_models()
        algo = change['new']
        
        if models[algo]:
            model_options = [f"{steps:,} steps" for steps, _ in models[algo][:15]]
            self.model_dropdown.options = model_options
        else:
            self.model_dropdown.options = [f'No {algo} models found']

    def update_model_info(self, change):
        """Update model information display"""
        model_text = change['new']
        algo = self.algo_dropdown.value
        
        if 'No' in model_text and 'found' in model_text:
            self.model_info.value = f'<div style="padding: 8px; background-color: #ffebee; border-radius: 3px; color: #c62828;">❌ No {algo} models available</div>'
            return
        
        try:
            steps = int(model_text.split()[0].replace(',', ''))
            model_name = f"{algo}_{steps}"
            model_path = f"models/{model_name}.zip"
            
            if os.path.exists(model_path):
                file_size = os.path.getsize(model_path) / (1024*1024)  # MB
                info_html = f'''
                <div style="padding: 8px; background-color: #e8f5e8; border-radius: 3px;">
                    <b>📊 Model Info:</b><br>
                    🏷️ Name: {model_name}<br>
                    🎯 Training Steps: {steps:,}<br>
                    💾 File Size: {file_size:.1f} MB<br>
                    📁 Path: {model_path}
                </div>
                '''
            else:
                info_html = f'<div style="padding: 8px; background-color: #ffebee; border-radius: 3px; color: #c62828;">❌ Model file not found: {model_path}</div>'
                
            self.model_info.value = info_html
        except:
            self.model_info.value = '<div style="padding: 8px; background-color: #fff3e0; border-radius: 3px;">⚠️ Invalid model selection</div>'

    def get_selected_model_name(self):
        """Get the selected model name"""
        model_text = self.model_dropdown.value
        algo = self.algo_dropdown.value
        
        if 'No' in model_text and 'found' in model_text:
            return None
        
        try:
            steps = int(model_text.split()[0].replace(',', ''))
            return f"{algo}_{steps}"
        except:
            return None

    def execute_command(self, button):
        """Execute the sb3.py command"""
        model_name = self.get_selected_model_name()
        if not model_name:
            self.status_label.value = '<div style="padding: 10px; background-color: #ffebee; border-radius: 5px; color: #c62828;"><b>❌ Error</b> - No valid model selected</div>'
            return
        
        algo = self.algo_dropdown.value
        env = self.env_dropdown.value
        model_path = f"models/{model_name}.zip"
        
        if not os.path.exists(model_path):
            self.status_label.value = f'<div style="padding: 10px; background-color: #ffebee; border-radius: 5px; color: #c62828;"><b>❌ Error</b> - Model file not found: {model_path}</div>'
            return
        
        # Update status
        self.status_label.value = f'<div style="padding: 10px; background-color: #fff3e0; border-radius: 5px; color: #f57c00;"><b>🟡 Running</b> - Executing {model_name} on {env}</div>'
        
        with self.output:
            clear_output(wait=True)
            
            print(f"🚀 Enhanced Mujoco Viewer - Execution Started")
            print(f"{'='*60}")
            print(f"📊 Model: {model_name}")
            print(f"🤖 Algorithm: {algo}")
            print(f"🎮 Environment: {env}")
            print(f"📁 Model Path: {model_path}")
            print(f"⏰ Started at: {time.strftime('%H:%M:%S')}")
            print(f"{'='*60}\n")
            
            try:
                cmd = ["python", "sb3.py", env, algo, "-s", model_path]
                print(f"🔧 Command: {' '.join(cmd)}")
                print(f"🎮 Opening Mujoco viewer...")
                print(f"⚠️  Viewer will open in a separate window")
                print(f"⏹️  Use the Stop button or close the window to end\n")
                
                # Execute with real-time output
                self.current_process = subprocess.Popen(
                    cmd,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    text=True,
                    bufsize=1,
                    universal_newlines=True
                )
                
                # Monitor output in real-time
                while True:
                    output = self.current_process.stdout.readline()
                    if output == '' and self.current_process.poll() is not None:
                        break
                    if output:
                        print(output.strip())
                
                rc = self.current_process.poll()
                if rc == 0:
                    print(f"\n✅ Execution completed successfully at {time.strftime('%H:%M:%S')}!")
                    self.status_label.value = '<div style="padding: 10px; background-color: #e8f5e8; border-radius: 5px; color: #2e7d32;"><b>🟢 Completed</b> - Execution finished successfully</div>'
                else:
                    print(f"\n❌ Execution failed with return code: {rc}")
                    self.status_label.value = f'<div style="padding: 10px; background-color: #ffebee; border-radius: 5px; color: #c62828;"><b>❌ Failed</b> - Return code: {rc}</div>'
                    
            except Exception as e:
                print(f"💥 Error: {str(e)}")
                self.status_label.value = f'<div style="padding: 10px; background-color: #ffebee; border-radius: 5px; color: #c62828;"><b>💥 Error</b> - {str(e)}</div>'
            finally:
                self.current_process = None

    def stop_execution(self, button):
        """Stop the current execution"""
        if self.current_process:
            try:
                self.current_process.terminate()
                self.status_label.value = '<div style="padding: 10px; background-color: #ffebee; border-radius: 5px; color: #c62828;"><b>⏹️ Stopped</b> - Execution terminated by user</div>'
                with self.output:
                    print(f"\n⏹️ Execution stopped by user at {time.strftime('%H:%M:%S')}")
            except:
                pass
            self.current_process = None
        else:
            self.status_label.value = '<div style="padding: 10px; background-color: #fff3e0; border-radius: 5px; color: #f57c00;"><b>⚠️ Info</b> - No active execution to stop</div>'

    def refresh_models(self, button):
        """Refresh the available models list"""
        models = self.get_available_models()
        algo = self.algo_dropdown.value
        
        if models[algo]:
            model_options = [f"{steps:,} steps" for steps, _ in models[algo][:15]]
            self.model_dropdown.options = model_options
            self.status_label.value = f'<div style="padding: 10px; background-color: #e8f5e8; border-radius: 5px; color: #2e7d32;"><b>🔄 Refreshed</b> - Found {len(models[algo])} {algo} models</div>'
        else:
            self.model_dropdown.options = [f'No {algo} models found']
            self.status_label.value = f'<div style="padding: 10px; background-color: #fff3e0; border-radius: 5px; color: #f57c00;"><b>⚠️ No Models</b> - No {algo} models found</div>'

    def display(self):
        """Display the enhanced UI"""
        # Custom header with gradient styling
        header_html = """
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    color: white; padding: 25px; text-align: center; 
                    border-radius: 12px; margin-bottom: 25px;
                    box-shadow: 0 8px 32px rgba(0,0,0,0.1);">
            <h1 style="margin: 0; font-size: 2.2em;">🚀 Enhanced Mujoco Viewer</h1>
            <p style="margin: 10px 0 0 0; font-size: 1.1em; opacity: 0.9;">Interactive interface for RL model visualization and control</p>
        </div>
        """
        display(HTML(header_html))
        
        # Model selection section
        selection_header = widgets.HTML('<h3 style="color: #333; margin: 20px 0 10px 0;">🤖 Model Selection</h3>')
        
        controls_row1 = widgets.HBox([
            self.algo_dropdown,
            self.model_dropdown,
            self.env_dropdown
        ], layout=widgets.Layout(margin='0 0 15px 0'))
        
        # Control buttons section
        controls_header = widgets.HTML('<h3 style="color: #333; margin: 20px 0 10px 0;">🎮 Controls</h3>')
        
        buttons_row = widgets.HBox([
            self.execute_btn,
            self.stop_btn,
            self.refresh_btn
        ], layout=widgets.Layout(margin='0 0 15px 0'))
        
        # Status and info section
        status_header = widgets.HTML('<h3 style="color: #333; margin: 20px 0 10px 0;">📊 Status & Information</h3>')
        
        info_section = widgets.VBox([
            self.status_label,
            self.model_info
        ], layout=widgets.Layout(margin='0 0 15px 0'))
        
        # Output section
        output_header = widgets.HTML('<h3 style="color: #333; margin: 20px 0 10px 0;">📺 Execution Output</h3>')
        
        # Main layout
        main_layout = widgets.VBox([
            selection_header,
            controls_row1,
            controls_header,
            buttons_row,
            status_header,
            info_section,
            output_header,
            self.output
        ])
        
        display(main_layout)
    
    def setup_ui(self):
        """Create the UI components"""
        models = self.get_available_models()
        
        # Algorithm dropdown
        self.algo_dropdown = widgets.Dropdown(
            options=['A2C', 'SAC', 'TD3'],
            value='A2C',
            description='🤖 Algorithm:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='200px')
        )
        
        # Model dropdown
        initial_models = [f"{steps:,} steps" for steps, _ in models['A2C'][:15]] if models['A2C'] else ['No models found']
        self.model_dropdown = widgets.Dropdown(
            options=initial_models,
            description='📊 Model:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='300px')
        )
        
        # Environment dropdown
        self.env_dropdown = widgets.Dropdown(
            options=['Humanoid-v4', 'Humanoid-v5', 'HumanoidStandup-v4'],
            value='Humanoid-v4',
            description='🎮 Environment:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='250px')
        )
        
        # Control buttons
        button_layout = widgets.Layout(width='160px', height='45px')
        
        self.execute_btn = widgets.Button(
            description='🚀 Run Viewer',
            button_style='primary',
            layout=button_layout,
            tooltip='Execute sb3.py with selected model'
        )
        
        self.stop_btn = widgets.Button(
            description='⏹️ Stop',
            button_style='danger',
            layout=button_layout,
            tooltip='Stop current execution'
        )
        
        self.refresh_btn = widgets.Button(
            description='🔄 Refresh Models',
            button_style='info',
            layout=button_layout,
            tooltip='Refresh available models list'
        )
        
        # Status display
        self.status_label = widgets.HTML(
            value='<div style="padding: 10px; background-color: #e3f2fd; border-radius: 5px;"><b>🟢 Ready</b> - Select a model and click Run Viewer</div>'
        )
        
        # Output area with enhanced styling
        self.output = widgets.Output(
            layout={
                'border': '2px solid #4CAF50', 
                'height': '450px', 
                'overflow': 'scroll',
                'padding': '10px',
                'background-color': '#fafafa'
            }
        )
        
        # Model info display
        self.model_info = widgets.HTML(
            value='<div style="padding: 8px; background-color: #f5f5f5; border-radius: 3px;">Select a model to see details</div>'
        )
        
        # Bind events
        self.algo_dropdown.observe(self.update_models, names='value')
        self.model_dropdown.observe(self.update_model_info, names='value')
        self.execute_btn.on_click(self.execute_command)
        self.stop_btn.on_click(self.stop_execution)
        self.refresh_btn.on_click(self.refresh_models)
        
        # Initial model info update
        self.update_model_info({'new': self.model_dropdown.value})

# Create the enhanced viewer instance
print("🎛️ Creating Enhanced Mujoco Viewer UI...")
enhanced_viewer = EnhancedMujocoViewer()
print("✅ UI created successfully!")


🎛️ Creating Enhanced Mujoco Viewer UI...
✅ UI created successfully!


In [6]:
# This cell is no longer needed - all methods are now in the main class definition above
print("✅ All methods are now properly integrated in the class definition!")


✅ All methods are now properly integrated in the class definition!


In [7]:
# This cell is no longer needed - all methods are now in the main class definition above
print("✅ All execution and control methods are now properly integrated!")


✅ All execution and control methods are now properly integrated!


In [8]:
# This cell is no longer needed - the display method is now in the main class definition above
print("✅ Display method is now properly integrated in the class!")


✅ Display method is now properly integrated in the class!


In [9]:
# Display the interactive UI dashboard
enhanced_viewer.display()
